In [1]:
from sklearn.utils import class_weight
from xgboost import XGBClassifier, cv
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import xgboost
from joblib import dump, load
from datetime import datetime
import spatial_efd

In [2]:
train_df = gpd.read_file("train.geojson", index_col=0)
test_df = gpd.read_file("test.geojson", index_col=0)

In [3]:
polygons=train_df['geometry']

In [8]:
# on train

def elliptic_coeffs_train(harmonics):
    l=list()
    l2=list()
    polygon=train_df['geometry']
    for i in range(len(train_df)):
        h=harmonics
        polygon_i=polygon.iloc[i]
        coords=list(polygon_i.boundary.coords)
        x=[z[0] for z in coords]
        y=[z[1] for z in coords]
        coeffs = spatial_efd.CalculateEFD(x, y,h)
        coeffs, _ = spatial_efd.normalize_efd(coeffs, size_invariant=True)
        coeffs=spatial_efd.AverageCoefficients(coeffs)[0]
        nyquist = spatial_efd.Nyquist(x)
        tmpcoeffs = spatial_efd.CalculateEFD(x, y, nyquist)
        fourier_power=spatial_efd.FourierPower(tmpcoeffs, x,0.99999)
        l2.append(fourier_power)
        l.append(coeffs)
    return np.array(l),l2

harmonics=3
df=list()
coeffs=elliptic_coeffs_train(harmonics=harmonics)[0]
powers=elliptic_coeffs_train(harmonics=harmonics)[1]
dic_pow={}
for k in range(len(train_df)):
    dic={}
    for j in range(4):
        dic[f"coeff{j+1}"]=coeffs[k,j]
    dic[f"power"]=int(powers[k]) if powers[k]!=None else None
    df.append(dic)
    

df=pd.DataFrame(df)

df.to_csv("fourier_coefficients_train.csv")



In [ ]:
# on test 

def elliptic_coeffs_test(harmonics):
    l=list()
    l2=list()
    polygon=test_df['geometry']
    for i in range(len(test_df)):
        h=harmonics
        polygon_i=polygon.iloc[i]
        coords=list(polygon_i.boundary.coords)
        x=[z[0] for z in coords]
        y=[z[1] for z in coords]
        coeffs = spatial_efd.CalculateEFD(x, y,h)
        coeffs, _ = spatial_efd.normalize_efd(coeffs, size_invariant=True)
        coeffs=spatial_efd.AverageCoefficients(coeffs)[0]
        nyquist = spatial_efd.Nyquist(x)
        tmpcoeffs = spatial_efd.CalculateEFD(x, y, nyquist)
        fourier_power=spatial_efd.FourierPower(tmpcoeffs, x,0.99999)
        l2.append(fourier_power)
        l.append(coeffs)
    return np.array(l),l2

harmonics=3
df=list()
coeffs=elliptic_coeffs_test(harmonics=harmonics)[0]
powers=elliptic_coeffs_test(harmonics=harmonics)[1]
dic_pow={}
for k in range(len(test_df)):
    dic={}
    for j in range(4):
        dic[f"coeff{j+1}"]=coeffs[k,j]
    dic[f"power"]=int(powers[k]) if powers[k]!=None else None
    df.append(dic)
    

df=pd.DataFrame(df)

df.to_csv("fourier_coefficients_test.csv")